In [1]:
from nltk.tokenize import sent_tokenize
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
from collections import Counter
import numpy as np

**Задание 1**

In [ ]:
with open('2ch_corpus.txt', 'r', encoding='utf-8') as f1, open('lenta.txt', 'r', encoding='utf-8') as f2:
    dvach = f1.read()[:400000]
    lenta = f2.read()[:400000]

К сожалению, даже на колабе на хватает памяти ни для полных корпусов, ни даже для их половины, поэтому пришлось урезать вот так. Из-за этого, конечно, пострадала творческая составляющая в генерации текстов (т.е. они, наверное, частично просто повторяют тексты из корпуса).

In [2]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

In [3]:
def ngrammer(tokens, n=3):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [ ]:
sentences_dvach = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach)]
sentences_news = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(lenta)]

In [ ]:
unigrams_dvach = Counter()
bigrams_dvach = Counter()
trigrams_dvach = Counter()

for sentence in sentences_dvach:
    unigrams_dvach.update(sentence)
    bigrams_dvach.update(ngrammer(sentence, n=2))
    trigrams_dvach.update(ngrammer(sentence))


unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence, n=2))
    trigrams_news.update(ngrammer(sentence))

In [ ]:
matrix_dvach = np.zeros((len(bigrams_dvach), len(unigrams_dvach)))

id2word_dvach = list(unigrams_dvach)
bi_id2word_dvach = list(bigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}
bi_word2id_dvach = {word:i for i, word in enumerate(bi_id2word_dvach)}

for ngram in trigrams_dvach:
    words12 = ' '.join(ngram.split()[:2])
    word3 = ngram.split()[2]
    matrix_dvach[bi_word2id_dvach[words12]][word2id_dvach[word3]] =  (trigrams_dvach[ngram]/
                                                                     bigrams_dvach[words12])

In [ ]:
matrix_news = np.zeros((len(bigrams_news), len(unigrams_news)))

id2word_news = list(unigrams_news)
bi_id2word_news = list(bigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}
bi_word2id_news = {word:i for i, word in enumerate(bi_id2word_news)}


# вероятность расчитываем точно также
for ngram in trigrams_news:
    words12 = ' '.join(ngram.split()[:2])
    word3 = ngram.split()[2]
    matrix_news[bi_word2id_news[words12]][word2id_news[word3]] =  (trigrams_news[ngram]/
                                                                     bigrams_news[words12])

In [4]:
def generate(matrix, id2word, bi_word2id, n=100, start='<start> <start>'):
    text = []
    current_idx = bi_word2id[start]
    current_bigram = start

    for i in range(n):
        
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        #print('следующее слово: ' + id2word[chosen])
        text.append(id2word[chosen])
        current_bigram = current_bigram.split()[1] + ' ' + id2word[chosen]

        if id2word[chosen] == '<end>':
            chosen = bi_word2id['<start> <start>']
            current_bigram = '<start> <start>'
        #print(current_bigram)
        current_idx = bi_word2id[current_bigram]
    return ' '.join(text)

In [ ]:
print(generate(matrix_dvach, id2word_dvach, bi_word2id_dvach).replace('<end>', '\n'))

спроси лучше в скорую есть смысл позвонить 
 2016 не иметь 
 аристотель бы сильно удивился если бы ты выкладывать в сеть видео на которых ты не выразишь всех ужасов войны но свои почему-то не предлагаешь 
 так как слишком мало уточняющих факторов специально для тебя в голове 
 слово кружок выдумали коммимрази 
 с бутылки встань сначала зачем вставать 
 казалось бы безопасной дистанции 
 ты сам в это говно 
 это не мешает это делать 
 неуместные метафоры и сбитый ритм создают ощущение нескладности произведения но это было бы если это я бессознательно захожу по несколько раз в


In [ ]:
print(generate(matrix_news, id2word_news, bi_word2id_news).replace('<end>', '\n'))

как сообщает риа новости 25 млн долларов было выделено шамилю басаеву частными лицами и общественными организациями происламского толка 
 основная причина распространения болезни отсутствие достаточных средств на предмет проблемы 2000 стран а также выработка неотложных мер по пресечению подобных действий организовав лишь оцепление омоном здания правительства на состоявшейся после встречи с руководством района и представителями мвд республики собравшиеся разблокировали дорогу и разошлись 
 пресс-служба увд приморского края сообщает что в здании никаких взрывных устройств и взрывчатых веществ который по каким-то причинам мог взорваться 
 наша боль безмерна 
 это — здоровый детина с типично прусским наглым лицом и мутными глазами


Я попробовала сгенерировать текст на основе корпуса сценариев Тарантино, но получилось как-то не очень( Наверное, дело в том, что их нужно сначала очистить от всей метаинформации типа авторских ремарок и имен персонажей.

**Задание 2**

In [44]:
import gensim
from razdel import sentenize
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
import re
from collections import Counter
import numpy as np

In [32]:
stops = set(stopwords.words('russian') + ["это", "весь"])
morph = MorphAnalyzer()

def normalize(text):
    tokens = re.findall('[а-яёa-z0-9]+', text.lower())
    normalized_text = [morph.parse(word)[0].normal_form for word \
                                                            in tokens]
    normalized_text = [word for word in normalized_text if len(word) > 2 and word not in stops]
    
    return normalized_text

def preprocess(text):
    sents = sentenize(text)
    return [normalize(sent.text) for sent in sents]

In [30]:
with open('lenta.txt', 'r', encoding='utf-8') as f:
  corpus = f.read()

corpus = preprocess(corpus)

In [94]:
def pmi_scorer(worda_count, wordb_count, bigram_count, len_vocab, min_count, corpus_word_count):
    try:
      pmi_score = np.log((bigram_count/corpus_word_count) / ((worda_count/corpus_word_count) * (wordb_count/corpus_word_count)))
    except ZeroDivisionError:
      return 0
      
    return pmi_score

Попробуем установить порог вхождения 0 и проведем последовательное обучение:

In [117]:
ph = gensim.models.Phrases(corpus, scoring=pmi_scorer, threshold=0.00)
p = gensim.models.phrases.Phraser(ph)
ph2 = gensim.models.Phrases(p[corpus])
p2 = gensim.models.phrases.Phraser(ph2)
p2[p[corpus[300]]]

['мера_расследование',
 'необходимо_сделать',
 'вывод_получить',
 'удовлетворительный_информация',
 'использование_прежний',
 'кредит_сказать',
 'саммерс_министр',
 'юстиция_юрий',
 'чайка_направить',
 'руководитель_территориальный',
 'орган_учреждение',
 'минюст_письмо',
 'который_качество',
 'главный_задача',
 'близкий_месяц',
 'обязать_организовать',
 'постоянный_контроль',
 'деятельность_политический',
 'общественный_объединение',
 'участвовать_выборы',
 'обратить_особый',
 'внимание_соответствие',
 'проводить_мероприятие',
 'уставный_цель']

Как и ожидалось, все нграммы проходят порог. Теперь повысим его до 5 и проверим результат на том же предложении:

In [119]:
ph = gensim.models.Phrases(corpus, scoring=pmi_scorer, threshold=5.00)
p = gensim.models.phrases.Phraser(ph)
ph2 = gensim.models.Phrases(p[corpus])
p2 = gensim.models.phrases.Phraser(ph2)
p2[p[corpus[300]]]

['мера',
 'расследование',
 'необходимо',
 'сделать_вывод',
 'получить',
 'удовлетворительный',
 'информация',
 'использование',
 'прежний',
 'кредит',
 'сказать',
 'саммерс',
 'министр_юстиция_юрий',
 'чайка',
 'направить',
 'руководитель',
 'территориальный_орган',
 'учреждение',
 'минюст',
 'письмо',
 'который',
 'качество',
 'главный_задача',
 'близкий_месяц',
 'обязать',
 'организовать',
 'постоянный',
 'контроль',
 'деятельность',
 'политический',
 'общественный_объединение',
 'участвовать_выборы',
 'обратить_особый',
 'внимание',
 'соответствие',
 'проводить',
 'мероприятие',
 'уставный_цель']

Уже лучше, но кажется, что некоторые сочетания (министр_юстиция_Юрий, обратить_особый) все-таки не совсем оправданы. Теперь попробуем поставить порог в 10, 20 и  - на всякий случай - 100:

In [121]:
ph = gensim.models.Phrases(corpus, scoring=pmi_scorer, threshold=10.00)
p = gensim.models.phrases.Phraser(ph)
ph2 = gensim.models.Phrases(p[corpus])
p2 = gensim.models.phrases.Phraser(ph2)
p2[p[corpus[300]]]

['мера',
 'расследование',
 'необходимо',
 'сделать_вывод',
 'получить',
 'удовлетворительный',
 'информация',
 'использование',
 'прежний',
 'кредит',
 'сказать',
 'саммерс',
 'министр_юстиция',
 'юрий_чайка',
 'направить',
 'руководитель',
 'территориальный_орган',
 'учреждение',
 'минюст',
 'письмо',
 'который',
 'качество',
 'главный_задача',
 'близкий',
 'месяц',
 'обязать',
 'организовать',
 'постоянный',
 'контроль',
 'деятельность',
 'политический_общественный',
 'объединение',
 'участвовать_выборы',
 'обратить',
 'особый_внимание',
 'соответствие',
 'проводить',
 'мероприятие',
 'уставный',
 'цель']

In [122]:
ph = gensim.models.Phrases(corpus, scoring=pmi_scorer, threshold=20.00)
p = gensim.models.phrases.Phraser(ph)
ph2 = gensim.models.Phrases(p[corpus])
p2 = gensim.models.phrases.Phraser(ph2)
p2[p[corpus[300]]]

['мера',
 'расследование',
 'необходимо',
 'сделать_вывод',
 'получить',
 'удовлетворительный',
 'информация',
 'использование',
 'прежний',
 'кредит',
 'сказать',
 'саммерс',
 'министр_юстиция',
 'юрий_чайка',
 'направить',
 'руководитель',
 'территориальный_орган',
 'учреждение',
 'минюст',
 'письмо',
 'который',
 'качество',
 'главный_задача',
 'близкий_месяц',
 'обязать',
 'организовать',
 'постоянный',
 'контроль',
 'деятельность',
 'политический_общественный',
 'объединение',
 'участвовать_выборы',
 'обратить',
 'особый_внимание',
 'соответствие',
 'проводить',
 'мероприятие',
 'уставный',
 'цель']

In [123]:
ph = gensim.models.Phrases(corpus, scoring=pmi_scorer, threshold=100.00)
p = gensim.models.phrases.Phraser(ph)
ph2 = gensim.models.Phrases(p[corpus])
p2 = gensim.models.phrases.Phraser(ph2)
p2[p[corpus[300]]]

['мера',
 'расследование',
 'необходимо',
 'сделать_вывод',
 'получить',
 'удовлетворительный',
 'информация',
 'использование',
 'прежний',
 'кредит',
 'сказать',
 'саммерс',
 'министр_юстиция',
 'юрий_чайка',
 'направить',
 'руководитель',
 'территориальный_орган',
 'учреждение',
 'минюст',
 'письмо',
 'который',
 'качество',
 'главный_задача',
 'близкий_месяц',
 'обязать',
 'организовать',
 'постоянный',
 'контроль',
 'деятельность',
 'политический_общественный',
 'объединение',
 'участвовать_выборы',
 'обратить',
 'особый_внимание',
 'соответствие',
 'проводить',
 'мероприятие',
 'уставный',
 'цель']

Странно, но, кажется, повышение порога не меняет результат. Скорее всего, это значит, что в моей функции, вычисляющей PMI, что-то пошло не так.